In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sns
#from tensorflow.keras.utils import get_file
from sklearn.metrics import roc_curve, auc, confusion_matrix
from imblearn.metrics import sensitivity_score, specificity_score
import os
import glob
import zipfile
import random
import numpy as np
import pandas as pd
import tensorflow.keras.utils as uti

In [ ]:
#具体的数据集，在实验室1070主机上，ip地址为129.105.15.197,我使用的是窗口号是47，以下代码中出现的地址都可以在该主机上找到对应处理好的数据集
#原始数据集就是蔡老师打包发过来的数据集，使用Data_Preprocess.ipynb文件处理后也可得到以下代码需要的数据集。

In [2]:
# to get consistent results after multiple runs
tf.random.set_seed(7)
np.random.seed(7)
random.seed(7)

# 0 for benign, 1 for malignant
dic, class_names = {}, []
# 这里，分成多少类，range就是多少+1
for i in range(16):
    class_names.append(str(i))
    dic[str(i)] = i

In [3]:
# preparing data
# generate CSV metadata file to read img paths and labels from it
def generate_csv(folder, label2int):
    folder_name = os.path.basename(folder)
    # convert comma separated labels into a list

    labels = list(label2int)
    # generate CSV file
    df = pd.DataFrame(columns=["filepath", "label"])
    i = 0
    for label in labels:
        print("Reading", os.path.join(folder, label, "*"))
        for filepath in glob.glob(os.path.join(folder, label, "*")):
            df.loc[i] = [filepath, label2int[label]]
            i += 1
    output_file = f"{folder_name}.csv"
    print("Saving", output_file)
    df.to_csv(output_file)

# generate CSV files for all data portions, labeling nevus and seborrheic keratosis
# as 0 (benign), and melanoma as 1 (malignant)
# you should replace "data" path to your extracted dataset path
# don't replace if you used download_and_extract_dataset() function
generate_csv("data/train", dic)
generate_csv("data/valid", dic)
generate_csv("data/test", dic)

Reading data/train/0/*
Reading data/train/1/*
Reading data/train/2/*
Reading data/train/3/*
Reading data/train/4/*
Reading data/train/5/*
Reading data/train/6/*
Reading data/train/7/*
Reading data/train/8/*
Reading data/train/9/*
Reading data/train/10/*
Reading data/train/11/*
Reading data/train/12/*
Reading data/train/13/*
Reading data/train/14/*
Reading data/train/15/*
Reading data/train/16/*
Reading data/train/17/*
Reading data/train/18/*
Reading data/train/19/*
Reading data/train/20/*
Reading data/train/21/*
Reading data/train/22/*
Reading data/train/23/*
Reading data/train/24/*
Reading data/train/25/*
Reading data/train/26/*
Reading data/train/27/*
Reading data/train/28/*
Reading data/train/29/*
Reading data/train/30/*
Reading data/train/31/*
Reading data/train/32/*
Reading data/train/33/*
Reading data/train/34/*
Reading data/train/35/*
Reading data/train/36/*
Reading data/train/37/*
Reading data/train/38/*
Reading data/train/39/*
Reading data/train/40/*
Reading data/train/41/*
Re

Reading data/test/108/*
Reading data/test/109/*
Reading data/test/110/*
Reading data/test/111/*
Reading data/test/112/*
Reading data/test/113/*
Reading data/test/114/*
Reading data/test/115/*
Reading data/test/116/*
Reading data/test/117/*
Reading data/test/118/*
Reading data/test/119/*
Saving test.csv


In [9]:
# loading data
train_metadata_filename = "train.csv"
valid_metadata_filename = "valid.csv"
# load CSV files as DataFrames
df_train = pd.read_csv(train_metadata_filename)
df_valid = pd.read_csv(valid_metadata_filename)
n_training_samples = len(df_train)
n_validation_samples = len(df_valid)
print("Number of training samples:", n_training_samples)
print("Number of validation samples:", n_validation_samples)
# train_ds = tf.data.Dataset.from_tensor_slices((df_train["filepath"], df_train["label"]))
# valid_ds = tf.data.Dataset.from_tensor_slices((df_valid["filepath"], df_valid["label"]))
train_ds = tf.data.Dataset.from_tensor_slices((df_train["filepath"], uti.to_categorical(df_train["label"], 120)))
valid_ds = tf.data.Dataset.from_tensor_slices((df_valid["filepath"], uti.to_categorical(df_valid["label"], 120)))

Number of training samples: 3504
Number of validation samples: 1016


In [14]:
# preprocess data
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [299, 299])


def process_path(filepath, label):
    # load the raw data from the file as a string
    img = tf.io.read_file(filepath)
    img = decode_img(img)
    return img, label


# test_ds = test_ds
valid_ds = valid_ds.map(process_path)
train_ds = train_ds.map(process_path)
for image, label in train_ds.take(1):
    print("Image shape:", image.shape)
    print("Label:", label.numpy())

TypeError: in user code:

    <ipython-input-4-6f2139ad4ac6>:13 process_path  *
        img = tf.io.read_file(filepath)
    /home/amy/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/gen_io_ops.py:574 read_file  **
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    /home/amy/anaconda3/lib/python3.8/site-packages/tensorflow/python/framework/op_def_library.py:492 _apply_op_helper
        raise TypeError("%s expected type of %s." %

    TypeError: Input 'filename' of 'ReadFile' Op has type float32 that does not match expected type of string.


In [11]:
# training parameters
batch_size = 64
optimizer = "rmsprop"

def prepare_for_training(ds, cache=True, batch_size=64, shuffle_buffer_size=1000):
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
    # shuffle the dataset
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    # Repeat forever
    ds = ds.repeat()
    # split to batches
    ds = ds.batch(batch_size)
    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

valid_ds = prepare_for_training(valid_ds, batch_size=batch_size, cache="valid-cached-data")
train_ds = prepare_for_training(train_ds, batch_size=batch_size, cache="train-cached-data")

In [141]:
#这个环节有时候会有奇怪的错误，如果报错了跳过也可以，不是必要的
batch = next(iter(valid_ds))

def show_batch(batch):
    plt.figure(figsize=(12,12))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(batch[0][n])
        plt.axis('off')


show_batch(batch)

InvalidArgumentError: Data type mismatch at component 0: expected string but got float.

In [12]:
# building the model
# InceptionV3 model & pre-trained weights
module_url = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

m = tf.keras.Sequential([
    hub.KerasLayer(module_url, output_shape=[2048], trainable=False),
    tf.keras.layers.Dense(units = 120, activation="softmax")
])

m.build([None, 299, 299, 3])
m.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
m.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 2048)              21802784  
_________________________________________________________________
dense_1 (Dense)              (None, 120)               245880    
Total params: 22,048,664
Trainable params: 245,880
Non-trainable params: 21,802,784
_________________________________________________________________


In [13]:
model_name = f"benign-vs-malignant_{batch_size}_{optimizer}"
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=os.path.join("logs", model_name))
# saves model checkpoint whenever we reach better weights
modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(model_name + "_{val_loss:.3f}.h5", save_best_only=True, verbose=1)

history = m.fit(train_ds, validation_data=valid_ds, 
                steps_per_epoch=n_training_samples // batch_size, 
                validation_steps=n_validation_samples // batch_size, verbose=1, epochs=100,
                callbacks=[tensorboard, modelcheckpoint])

Epoch 1/100


InvalidArgumentError:  Incompatible shapes at component 0: expected [?,200,200,3] but got [64,299,299,3].
	 [[node IteratorGetNext (defined at <ipython-input-13-f227e49934aa>:6) ]] [Op:__inference_train_function_55074]

Function call stack:
train_function


In [10]:
# evaluation
# load testing set
test_metadata_filename = "test.csv"
df_test = pd.read_csv(test_metadata_filename)
n_testing_samples = len(df_test)
print("Number of testing samples:", n_testing_samples)
test_ds = tf.data.Dataset.from_tensor_slices((df_test["filepath"], df_test["label"]))

def prepare_for_testing(ds, cache=True, shuffle_buffer_size=1000):
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    return ds

test_ds = test_ds.map(process_path)
test_ds = prepare_for_testing(test_ds, cache="test-cached-data")

# convert testing set to numpy array to fit in memory (don't do that when testing
# set is too large)
y_test = np.zeros((n_testing_samples,))
X_test = np.zeros((n_testing_samples, 299, 299, 3))
for i, (img, label) in enumerate(test_ds.take(n_testing_samples)):
    # print(img.shape, label.shape)
    X_test[i] = img
    y_test[i] = label.numpy()

print("y_test.shape:", y_test.shape)


Number of testing samples: 1200
y_test.shape: (1200,)


In [11]:
def partition(array, start, end):
    pivot = array[start][0]
    low, high = start+1, end
    while True:
        while low <= high and array[high][0] >= pivot:
            high -= 1
        while low <= high and array[low][0] <= pivot:
            low += 1
        if low <= high:
            array[low], array[high] = array[high], array[low]
        else:
            break
    array[start], array[high] = array[high], array[start]
    return high
    
def selecttopn(n, data, start, end):
    if start >= end:
        return set([ele[1] for ele in data[len(data)-n:]])
    p = partition(data, start, end)
    if len(data) - p == n:
        return set([ele[1] for ele in data[p:]])
    if len(data) - p < n:
        return selecttopn(n, data, start, p-1)
    else:
        return selecttopn(n, data, p+1, end)
    

In [22]:
'''test
data = [(0.5, 1), (0.4,2),(0.1,3),(0.2,4),(0.6,5),(0.9,6),(0.7,7)]
selecttopn(4,data,0,len(data)-1)
'''

{1, 5, 6, 7}

In [12]:
print("Evaluating the model...")
res = m.predict(X_test)
print("Finish!")

Evaluating the model...


In [15]:
# load the weights with the least loss
#m.load_weights("benign-vs-malignant_64_rmsprop_0.385.h5")
def get_acc(n):
    correct = 0
    for i,row in enumerate(res):
        row = list(zip(list(row),list(range(120))))
        top = selecttopn(n,row,0,len(row)-1)
        if y_test[i] in top:
            correct += 1
    print('========================================')
    print('Top',n)
    print('Accuracy:', correct/len(res),"\n")

for n in [1,3,5,10,20]:
    get_acc(n)
    

Top 1
Accuracy: 0.05 

Top 3
Accuracy: 0.19666666666666666 

Top 5
Accuracy: 0.32666666666666666 

Top 10
Accuracy: 0.5108333333333334 

Top 20
Accuracy: 0.6658333333333334 



In [ ]:
'''
loss, accuracy = m.evaluate(X_test, y_test, verbose=0)
print("Loss:", loss, "  Accuracy:", accuracy)
'''

In [17]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
model = InceptionV3(include_top=True,)

In [13]:
def get_predictions(threshold=None):
    """
    Returns predictions for binary classification given `threshold`
    For instance, if threshold is 0.3, then it'll output 1 (malignant) for that sample if
    the probability of 1 is 30% or more (instead of 50%)
    """
    y_pred = m.predict(X_test)
    if not threshold:
        threshold = 0.5
    result = np.zeros((n_testing_samples,))
    for i in range(n_testing_samples):
    # test melanoma probability
        if y_pred[i][0] >= threshold:
            result[i] = 1
    # else, it's 0 (benign)
    return result

threshold = 0.23
# get predictions with 23% threshold
# which means if the model is 23% sure or more that is malignant,
# it's assigned as malignant, otherwise it's benign
y_pred = get_predictions(threshold)

In [4]:
#测试GPU是否在运转
tf.config.list_physical_devices('GPU')

[]